In [441]:

import pandas as pd 


Pto = pd.read_csv('gs://beam-core-outputs/sfbay-baseline-20230526/postprocessOutputs/year-2020-iteration-4/personToVehicles.csv', nrows = 1000, index_col = 0)
Plans = pd.read_csv('gs://beam-core-outputs/sfbay-baseline-20230526/beam/year-2020-iteration-4/plans.csv.gz', nrows = None,)
Pt = pd.read_csv('gs://beam-core-outputs/sfbay-baseline-20230526/beam/year-2020-iteration-4/ITERS/it.0/0.events.csv.gz', nrows = None,
                usecols=['time', 'departureTime', 'vehicleType'])


Pto = Pto[Pto['mode']=='car']
Plans['planElementIndex'] -= 1
Plans = Plans[Plans['planElementType']=='activity']
Plans = Plans[Plans['planSelected']==True]
Plans = Plans.drop(columns = ['planIndex'])


In [509]:
def addGeometryIdToDataFrame(df, gdf, xcol, ycol, idColumn="geometry", df_geom='epsg:4326'): 
    gdf.crs = {'init': 'epsg:4326'}
    gdf_data = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df[xcol], df[ycol]))
    gdf_data.crs = {'init': df_geom}
    joined = gpd.sjoin(gdf_data.to_crs('epsg:26910'), gdf.to_crs('epsg:26910'))
    gdf_data = gdf_data.merge(joined['blkgrpid'], left_index=True, right_index=True, how="left")
    gdf_data.rename(columns={'blkgrpid': idColumn}, inplace=True)
    df = pd.DataFrame(gdf_data.drop(columns='geometry'))
    # df.drop(columns=[xcol, ycol], inplace=True)
    print(len(df))
    print(len(df.loc[~df.index.duplicated(keep='first'), :]))

    return df.loc[~df.index.duplicated(keep='first'), :]

In [470]:
pop = pd.read_csv('gs://beam-core-outputs/sfbay-baseline-20230526/activitysim/year-2020-iteration-4/persons.csv.gz', nrows = None,)
hs = pd.read_csv('gs://beam-core-outputs/sfbay-baseline-20230526/activitysim/year-2020-iteration-4/households.csv.gz', nrows = None,)

In [523]:
# Drop duplicates from Pto
Pto_unique = Pto.drop_duplicates(subset=['personID', 'planIndex'])

# Create a key in both dataframes for merging
Pto_unique['key'] = Pto_unique['personID'].astype(str) + '_' + Pto_unique['planIndex'].astype(str)
Plans['key'] = Plans['personId'].astype(str) + '_' + Plans['planElementIndex'].astype(str)

# Filter Plans based on the keys present in Pto_unique
filtered_plans = Plans[Plans['key'].isin(Pto_unique['key'])]

# Optionally, you can drop the 'key' column after filtering
filtered_plans = filtered_plans.drop(columns=['key'])

Pt['PathTraversalID'] = Pt.index
selected_columns = Pt[['PathTraversalID', 'time', 'departureTime', 'vehicleType']]
merged_Pto = Pto.merge(selected_columns, left_on='pathTraversalID', right_on='PathTraversalID', how='left')


print(len(filtered_plans),'Number of activity plans')
print(len(Pto_unique),'Number of car Pto')

/tmp/ipykernel_3374612/4010220988.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Pto_unique['key'] = Pto_unique['personID'].astype(str) + '_' + Pto_unique['planIndex'].astype(str)


353 Number of activity plans
353 Number of car Pto


In [524]:
# Rename columns in Plans or Pto
rename_dict = {
    'personId': 'personID',
    'planElementIndex': 'planIndex',
    # ... add more columns to rename as needed
}
filtered_plans = filtered_plans.rename(columns=rename_dict)
filtered_plans = filtered_plans[['personID','planIndex','activityEndTime','activityType']]

filtered_plans['activityEndTime'] = filtered_plans['activityEndTime'].replace(-np.inf, 24*3600)


combined_df = pd.concat([merged_Pto, filtered_plans], ignore_index=True, sort=False)


/opt/conda/lib/python3.10/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
/tmp/ipykernel_3374612/3841419929.py:1: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).

In [525]:
import pandas as pd
import numpy as np
import geopandas as gpd
# Sample dataframe
data = combined_df
df = pd.DataFrame(data)

# Create time bins
time_bins = pd.date_range("00:00", "23:59", freq="15T").time
seconds_list = [(time.hour * 3600 + time.minute * 60 + time.second) for time in time_bins]

def compute_bin_values(group):
    print('#####NEW GROUP####')
    
    print('DEPART \n',group['departureTime'])
    print('ARRIVE \n',group['departureTime']+group['duration'])
    print('ACTEND \n',group['activityEndTime'])
    
    total_length = group['length'].sum()
    total_duration = group['duration'].sum()
    total_speed = total_length/total_duration
    total_energy= group['primaryFuel'].sum()+group['secondaryFuel'].sum()
    total_energy_sec = total_energy/total_duration
    
    group = group.sort_values('departureTime')
    bgid_start = group.iloc[0]['bgid_start']
    juris_name_start = group.iloc[0]['juris_name_start']
    county_name_start = group.iloc[0]['county_name_start']
    tractid_start = group.iloc[0]['tractid_start']
    mpo_start = group.iloc[0]['mpo_start']

    bgid_end = group.iloc[-2]['bgid_end']
    juris_name_end = group.iloc[-2]['juris_name_end']
    county_name_end = group.iloc[-2]['county_name_end']
    tractid_end = group.iloc[-2]['tractid_end']
    mpo_end = group.iloc[-2]['mpo_end']
    
    veh_type = group.iloc[0]['vehicleType']

    row_test = group.iloc[0]
    row_test2 = group.iloc[-1]
    
    start_time_act = group.iloc[-2]['departureTime']+group.iloc[-2]['duration']
    end_time_act = group.iloc[-1]['activityEndTime']
    # row1 = group.iloc[1]
    # row2 = group.iloc[2]
    
    output = {}
    # Add the additional columns you mentioned
    additional_columns = {
        ('Person ID', '-'): int(row_test['personID']), 
        # ('Vehicle ID', '-'): int(row_test['vehicleID']), 
        ('Vehicle Type', '-'): veh_type, 
        ('Plan Index', '-'): int(row_test['planIndex']), 
        ('Activity Type', '-'): str(row_test2['activityType']), 
        ('Trip Length [m]', '-'): int(total_length), 
        ('Trip Duration [s]', '-'): int(total_duration), 
        ('Trip average Speed [m/s]', '-'): "{:.2f}".format(total_speed), 
        ('Trip Energy [J]', '-'): int(total_energy), 
        ('BGID start', '-'): bgid_start,
        ('TRACTID start', '-'): tractid_start,
        ('JURIS start', '-'): juris_name_start,
        ('COUNTY start', '-'): county_name_start,
        ('MPO start', '-'): mpo_start,
        ('TRACTID end', '-'): bgid_end,
        ('TRACTID end', '-'): tractid_end,
        ('JURIS end', '-'): juris_name_end,
        ('COUNTY end', '-'): county_name_end,
        ('MPO end', '-'): mpo_end
    }
    output.update(additional_columns)
    
    for time in time_bins:
        output[(time, 'Distance [m]')] = 0
        output[(time, 'Duration [s]')] = 0
        output[(time, 'Speed [m/s]')] = "{:.2f}".format(0)
        output[(time, 'Energy [J]')] = 0
        output[(time, 'Driving')] = 0
        output[(time, 'Parked')] = 0
    
    for index, row in group.iloc[:-1].iterrows():
    
        start_time = None
        end_time = None

        try:
            start_time = row['departureTime']
            end_time = row['departureTime']+row['duration']
        except:
            print('no time fo vehicle',row['vehicleID'])
            
        if start_time and end_time:
            # energy = row['primaryFuel'] + row['secondaryFuel'] /1000000000000

            for bin_time, sec_time in zip(time_bins, seconds_list):
                next_bin_time = sec_time + 900
                if sec_time <= start_time < next_bin_time or sec_time < end_time <= next_bin_time or start_time <= sec_time < end_time:
                    print('START  ', 'END    ','BIN ', 'NEXT_BIN'  )
                    print(start_time, end_time,sec_time, next_bin_time  )

                    fraction = 1 
                    if next_bin_time > end_time:
                        fraction-=((next_bin_time-end_time)/900)
                    if start_time > sec_time:
                        fraction-=(start_time-sec_time)/900 
                        
                    print('FRACTION',fraction)
                    
                    #REDUCE LENGTH TO THE 15' BIN MAYBE BASED ON AVERAGE SPEED
                    #split duration drive and duration parked
                    #split parking?
                    
                    output[(bin_time, 'Distance [m]')] += int(900 * fraction * total_speed)
                    output[(bin_time, 'Duration [s]')] += int(900 * fraction)
                    output[(bin_time, 'Speed [m/s]')] = "{:.2f}".format(total_speed)
                    output[(bin_time, 'Energy [J]')] += int(total_energy_sec * 900 * fraction)
                    output[(bin_time, 'Driving')] = 1
                    
    start_time = start_time_act 
    end_time = end_time_act
    
    if start_time and end_time:

        for bin_time, sec_time in zip(time_bins, seconds_list):
            next_bin_time = sec_time + 900
            if (sec_time <= start_time < next_bin_time or sec_time < end_time <= next_bin_time or start_time <= sec_time < end_time) and start_time < end_time:
                print(start_time, end_time,sec_time, next_bin_time  )

                fraction = 1 
                if next_bin_time > end_time:
                    fraction-=(next_bin_time-end_time)/900
                if start_time > sec_time:
                    fraction-=(start_time-sec_time)/900

                print('FRACTION',fraction)

                output[(bin_time, 'Duration [s]')] += int(900 * fraction)
                output[(bin_time, 'Parked')] = 1
    else:
        print('WARNING:, NO START OR END TIMES FOR ACTIVITY')

    
    
    return pd.Series(output)

# Apply the function to each pair of rows (trip) based on 'personID' and 'planIndex'
new_df = df.groupby(['personID', 'planIndex']).apply(compute_bin_values).reset_index(drop=True)

print(new_df)


#####NEW GROUP####
DEPART 
 498     22708.0
499     23739.0
1048        NaN
Name: departureTime, dtype: float64
ARRIVE 
 498     23739.0
499     23743.0
1048        NaN
dtype: float64
ACTEND 
 498         NaN
499         NaN
1048    59950.8
Name: activityEndTime, dtype: float64
START   END     BIN  NEXT_BIN
22708.0 23739.0 22500 23400
FRACTION 0.768888888888889
START   END     BIN  NEXT_BIN
22708.0 23739.0 23400 24300
FRACTION 0.3766666666666667
START   END     BIN  NEXT_BIN
23739.0 23743.0 23400 24300
FRACTION 0.0044444444444444176
23743.0 59950.8 23400 24300
FRACTION 0.6188888888888888
23743.0 59950.8 24300 25200
FRACTION 1
23743.0 59950.8 25200 26100
FRACTION 1
23743.0 59950.8 26100 27000
FRACTION 1
23743.0 59950.8 27000 27900
FRACTION 1
23743.0 59950.8 27900 28800
FRACTION 1
23743.0 59950.8 28800 29700
FRACTION 1
23743.0 59950.8 29700 30600
FRACTION 1
23743.0 59950.8 30600 31500
FRACTION 1
23743.0 59950.8 31500 32400
FRACTION 1
23743.0 59950.8 32400 33300
FRACTION 1
23743.0 59950.8

In [526]:
new_df

Person ID      Vehicle Type Plan Index Activity Type Trip Length [m]  \
            -                 -          -             -               -   
0       92712  cv-L1-17000-2019          1          work           25509   
1       92712  cv-L1-17000-2019          3        social           14623   
2       92712  cv-L1-17000-2019          5          Home           11680   
3       92713  cv-L1-17000-2019          3        atwork           53100   
4       92713  cv-L1-17000-2019          5          Work            2382   
..        ...               ...        ...           ...             ...   
348   6645556  cv-L1-17000-2019          9        eatout           11799   
349   6645556  cv-L1-17000-2019         11          Home           10866   
350   6645559  cv-L1-17000-2019          1        school            6069   
351   6713845  cv-L1-17000-2019          1      shopping            3792   
352   6713845  cv-L1-17000-2019          3          Home            3992   

    Trip Duration [s] Trip average Speed [m/s] Trip Energy [J]    BGID start  \
                    -                        -               -             -   
0                1035                    24.65        52645654  6.001408e+10   
1                1833                     7.98        48581486  6.075012e+10   
2                 507                    23.04        24442676  6.001403e+10   
3                2385                    22.26       113152030  6.001409e+10   
4                 115                    20.72         5045866  6.081610e+10   
..                ...                      ...             ...           ...   
348               893                    13.21        33807536  6.097152e+10   
349               615                    17.67        25328727  6.097151e+10   
350               302                    20.10        12625821  6.097153e+10   
351               178                    21.31         8629693  6.097153e+10   
352               228                    17.51         9565356  6.097153e+10   

    TRACTID start  ...    23:30:00                               23:45:00  \
                -  ... Speed [m/s] Energy [J] Driving Parked Distance [m]   
0    6.001408e+09  ...        0.00          0       0      0            0   
1    6.075012e+09  ...        0.00          0       0      0            0   
2    6.001403e+09  ...        0.00          0       0      1            0   
3    6.001409e+09  ...        0.00          0       0      0            0   
4    6.081610e+09  ...        0.00          0       0      0            0   
..            ...  ...         ...        ...     ...    ...          ...   
348  6.097152e+09  ...        0.00          0       0      0            0   
349  6.097151e+09  ...        0.00          0       0      1            0   
350  6.097153e+09  ...        0.00          0       0      0            0   
351  6.097153e+09  ...        0.00          0       0      0            0   
352  6.097153e+09  ...        0.00          0       0      1            0   

                                                        
    Duration [s] Speed [m/s] Energy [J] Driving Parked  
0              0        0.00          0       0      0  
1              0        0.00          0       0      0  
2            900        0.00          0       0      1  
3              0        0.00          0       0      0  
4              0        0.00          0       0      0  
..           ...         ...        ...     ...    ...  
348            0        0.00          0       0      0  
349          900        0.00          0       0      1  
350            0        0.00          0       0      0  
351            0        0.00          0       0      0  
352          900        0.00          0       0      1  

[353 rows x 593 columns]

In [517]:
pop_filtered = pop[pop.person_id.isin(new_df[('Person ID','-')])]
merged_df = pd.merge(pop_filtered, hs, left_on='household_id', right_on='household_id', how='left')
BGs = gpd.read_file('/vsicurl/https://github.com/LBNL-UCB-STI/beam-core-analysis/raw/main/Users/Zach/scenario/sfbay-blockgroups-2010/641aa0d4-ce5b-4a81-9c30-8790c4ab8cfb202047-1-wkkklf.j5ouj.shp')
        
merged_df = addGeometryIdToDataFrame(merged_df, BGs, 'home_x', 'home_y', 'homeBlockGroup')

block_info = pd.read_csv('https://github.com/LBNL-UCB-STI/beam-core-analysis/raw/main/Users/Nazanin/JoeFish_BlockGroup_Labels/bg_w_geog_labels.csv')
merged_df['homeBlockGroup'] = merged_df['homeBlockGroup'].fillna(0).astype(int)
merged_df = pd.merge(merged_df, block_info,  how='left',  left_on = 'homeBlockGroup',right_on = 'bgid')

merged_df.to_csv('outputs/pop_hs_database.csv')

/opt/conda/lib/python3.10/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/opt/conda/lib/python3.10/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


106
106


In [518]:
merged_df

,person_id,age,earning,edu,hispanic,hours,PNUM,race_id,relate,sex,...,num_travel_active_non_preschoolers,participates_in_jtf_model,joint_tour_frequency,num_hh_joint_tours,homeBlockGroup,bgid,tractid,juris_name,county_name,mpo
0,4568299,31,11400.0,21.0,0,20.0,1,6,0,2,...,2,True,0_tours,0,60855043154,60855043154,6085504315,San Jose,Santa Clara County,San Francisco Bay Area (MTC)
1,5049766,30,51000.0,21.0,0,42.0,2,1,1,1,...,2,True,0_tours,0,60855066052,60855066052,6085506605,San Jose,Santa Clara County,San Francisco Bay Area (MTC)
2,5049768,30,65000.0,22.0,0,70.0,1,6,0,2,...,2,True,0_tours,0,60855066052,60855066052,6085506605,San Jose,Santa Clara County,San Francisco Bay Area (MTC)
3,5754153,76,1100.0,22.0,0,5.0,1,1,0,2,...,1,False,0_tours,0,60855085041,60855085041,6085508504,Sunnyvale,Santa Clara County,San Francisco Bay Area (MTC)
4,6645556,25,30000.0,18.0,0,40.0,2,1,1,2,...,2,True,0_tours,0,60971517004,60971517004,6097151700,Santa Rosa,Sonoma County,San Francisco Bay Area (MTC)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101,1638782,43,0.0,21.0,0,0.0,1,1,0,2,...,5,True,1_Disc,1,60133451014,60133451014,6013345101,San Ramon,Contra Costa County,San Francisco Bay Area (MTC)
102,3290023,76,0.0,22.0,0,0.0,1,1,0,2,...,2,True,0_tours,0,60750426023,60750426023,6075042602,San Francisco,San Francisco County,San Francisco Bay Area (MTC)
103,3290025,86,0.0,21.0,0,0.0,2,1,15,2,...,2,True,0_tours,0,60750426023,60750426023,6075042602,San Francisco,San Francisco County,San Francisco Bay Area (MTC)
104,3840096,49,4500.0,21.0,0,35.0,2,1,1,2,...,5,True,0_tours,0,60816103043,60816103043,6081610304,Redwood City,San Mateo County,San Francisco Bay Area (MTC)


In [ ]:
#REFINE columns format

In [ ]:
#Carpool

In [ ]:
#ADD person attributes, already read pop and hs

In [ ]:
#Pay attention to trip after midnight

In [527]:
new_df.to_csv('new_df.csv')

In [439]:
new_df

Person ID Vehicle ID Plan Index Trip Length [m] Trip Duration [s]  \
            -          -          -               -                 -   
0       92712     100122          1           25509              1035   
1       92712     100122          3           14623              1833   
2       92712     100122          5           11680               507   
3       92713     100121          3           53100              2385   
4       92713     100121          5            2382               115   
..        ...        ...        ...             ...               ...   
348   6645556     100158          9           11799               893   
349   6645556     100158         11           10866               615   
350   6645559     100159          1            6069               302   
351   6713845     100101          1            3792               178   
352   6713845     100101          3            3992               228   

    Trip average Speed [m/s] Trip Energy [J]    BGID start TRACTID start  \
                           -               -             -             -   
0                      24.65        52645654  6.001408e+10  6.001408e+09   
1                       7.98        48581486  6.075012e+10  6.075012e+09   
2                      23.04        24442676  6.001403e+10  6.001403e+09   
3                      22.26       113152030  6.001409e+10  6.001409e+09   
4                      20.72         5045866  6.081610e+10  6.081610e+09   
..                       ...             ...           ...           ...   
348                    13.21        33807536  6.097152e+10  6.097152e+09   
349                    17.67        25328727  6.097151e+10  6.097151e+09   
350                    20.10        12625821  6.097153e+10  6.097153e+09   
351                    21.31         8629693  6.097153e+10  6.097153e+09   
352                    17.51         9565356  6.097153e+10  6.097153e+09   

       JURIS start  ...    23:30:00                               23:45:00  \
                 -  ... Speed [m/s] Energy [J] Driving Parked Distance [m]   
0          Oakland  ...        0.00          0       0      0            0   
1    San Francisco  ...        0.00          0       0      0            0   
2          Oakland  ...        0.00          0       0      1            0   
3          Oakland  ...        0.00          0       0      0            0   
4     Redwood City  ...        0.00          0       0      0            0   
..             ...  ...         ...        ...     ...    ...          ...   
348     Santa Rosa  ...        0.00          0       0      0            0   
349            NaN  ...        0.00          0       0      1            0   
350            NaN  ...        0.00          0       0      0            0   
351            NaN  ...        0.00          0       0      0            0   
352     Santa Rosa  ...        0.00          0       0      1            0   

                                                        
    Duration [s] Speed [m/s] Energy [J] Driving Parked  
0              0        0.00          0       0      0  
1              0        0.00          0       0      0  
2            900        0.00          0       0      1  
3              0        0.00          0       0      0  
4              0        0.00          0       0      0  
..           ...         ...        ...     ...    ...  
348            0        0.00          0       0      0  
349          900        0.00          0       0      1  
350            0        0.00          0       0      0  
351            0        0.00          0       0      0  
352          900        0.00          0       0      1  

[353 rows x 592 columns]